In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
import numpy as np
print(np.__version__)

1.23.5


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt  # Import Matplotlib for visualization

# Define the paths to the input files
data_dir = '/kaggle/input/optiver-trading-at-the-close/'

train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'example_test_files/test.csv')
sample_submission_file = os.path.join(data_dir, 'example_test_files/sample_submission.csv')
revealed_targets_file = os.path.join(data_dir, 'example_test_files/revealed_targets.csv')

# Step 1: Data Preparation
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
sample_submission = pd.read_csv(sample_submission_file)
revealed_targets = pd.read_csv(revealed_targets_file)

# Step 2: Preprocess Data
train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

# Step 3: Feature Engineering
def calculate_synthetic_index(data):
    if 'synthetic_index' in data.columns:
        data['synthetic_index'] = ((data['wap'].shift(-60) / data['wap'])) * 10000
    return data

if 'synthetic_index' in train_data.columns:
    train_data = train_data.groupby(['stock_id', 'date_id']).apply(calculate_synthetic_index)
    # Drop the 'synthetic_index' column
    train_data = train_data.drop(columns=['synthetic_index'])

if 'synthetic_index' in test_data.columns:
    test_data = test_data.groupby(['stock_id', 'date_id']).apply(calculate_synthetic_index)
    # Drop the 'synthetic_index' column
    test_data = test_data.drop(columns=['synthetic_index'])

# Step 4: Model Selection
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Step 5: Training the Model
X = train_data.drop(['target'], axis=1)
y = train_data['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_valid = imputer.transform(X_valid)

model.fit(X_train, y_train)

# Step 6: Prediction
predictions = model.predict(X_valid)

# Step 7: Visualize the Model's Output
plt.figure(figsize=(12, 6))
plt.plot(predictions, label='Predicted')
plt.xlabel('Sample Index')
plt.ylabel('Predicted Value')
plt.title('Model Predictions')
plt.legend()
plt.show()


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
